In [1]:
import pandas as pd
# 库文件
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from skopt import gp_minimize
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from scipy.optimize import minimize
from sklearn.metrics.pairwise import rbf_kernel
import pandas as pd
import tensorflow as tf
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties
# 设置中文字体
font = FontProperties(fname=r"c:\windows\fonts\simsun.ttc", size=12)  # 替换为你的中文字体文件路径


In [3]:
# 替换成你的Excel文件路径\
file_path = f'C:\\Users\\haokw\\Documents\\GitHub\\gaolu\\MPC\\高炉\\0数据\\高炉数据原件 - 副本.xlsx'
df1 = pd.read_excel(file_path,sheet_name='铁水质量')
print('1')


1


In [4]:
df = pd.concat([df1],axis=0)
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16252 entries, 0 to 16251
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   化验时间    16252 non-null  datetime64[ns]
 1   P       16252 non-null  float64       
 2   S       16252 non-null  float64       
 3   MN      16252 non-null  float64       
 4   SI      16252 non-null  float64       
 5   V       16252 non-null  float64       
 6   TI      16252 non-null  float64       
 7   As      16252 non-null  float64       
dtypes: datetime64[ns](1), float64(7)
memory usage: 1015.9 KB


In [22]:
# 使用rename方法修改列名
df_params = df.copy()
df_params.rename(columns={'化验时间': '主参数时间戳'}, inplace=True)
df_processed = pd.DataFrame()


In [23]:
# 分批处理转成时间按戳
total_entries = len(df_params)
num_chunks = total_entries // 100000 + 1
for i in range(num_chunks):
    start_idx = i * 100000
    end_idx = min((i + 1) * 100000, total_entries)
    
    # 获取子集
    subset_df = df_params.iloc[start_idx:end_idx]

    # 定义起始时间
    start_time ='2023-02-09 00:00:00'
    start_time_stamp = pd.Timestamp(start_time).timestamp()

    # 转换为时间戳并保存到新列
    params_timestamp  = subset_df.iloc[:, 0].apply(lambda x: pd.Timestamp(x).timestamp())

    # 在后续步骤中使用新列进行进一步的操作
    sampling_time = 60
    subset_df['min'] = (params_timestamp - start_time_stamp) / sampling_time
    df_processed = pd.concat([df_processed,subset_df],axis=0)
    print(i,num_chunks)


0 1


C:\Users\haokw\AppData\Local\Temp\ipykernel_9564\1403589422.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_df['min'] = (params_timestamp - start_time_stamp) / sampling_time


In [26]:
df_processed.head(10)


,主参数时间戳,P,S,MN,SI,V,TI,As,min
0,2023-02-09 14:50:23,0.158,0.020,1.185,1.102,0.019,0.129,0.002,890.383333
1,2023-02-09 14:59:08,0.169,0.019,1.160,1.041,0.018,0.129,0.002,899.133333
2,2023-02-09 15:09:29,0.152,0.019,1.170,1.060,0.018,0.129,0.002,909.483333
3,2023-02-09 15:39:08,0.156,0.020,1.054,1.272,0.017,0.137,0.002,939.133333
4,2023-02-09 16:59:58,0.175,0.019,1.016,1.316,0.017,0.143,0.001,1019.966667
5,2023-02-09 17:03:58,0.148,0.016,0.985,1.361,0.016,0.138,0.002,1023.966667
6,2023-02-09 17:30:58,0.172,0.021,0.988,1.135,0.014,0.123,0.001,1050.966667
7,2023-02-09 17:54:06,0.171,0.015,1.007,1.222,0.016,0.135,0.002,1074.100000
8,2023-02-09 19:02:48,0.166,0.016,0.948,1.396,0.016,0.143,0.001,1142.800000
9,2023-02-09 19:25:05,0.164,0.017,0.942,1.282,0.016,0.131,0.003,1165.083333


In [28]:

print(df_processed)
new_sequence_1h_mean = df_processed.copy()
# 将 df_combined 添加到新的 ExcelWriter 中
with pd.ExcelWriter(f"C:\\Users\\haokw\\Documents\\GitHub\\gaolu\\MPC\\高炉\\0数据处理\\新输入输出模式\\new\\Si.xlsx", engine='xlsxwriter') as writer:
    # 将其他 sheet 写入新文件
    new_sequence_1h_mean.to_excel(writer, index=False)


                   主参数时间戳      P      S     MN     SI      V     TI     As  \
0     2023-02-09 14:50:23  0.158  0.020  1.185  1.102  0.019  0.129  0.002   
1     2023-02-09 14:59:08  0.169  0.019  1.160  1.041  0.018  0.129  0.002   
2     2023-02-09 15:09:29  0.152  0.019  1.170  1.060  0.018  0.129  0.002   
3     2023-02-09 15:39:08  0.156  0.020  1.054  1.272  0.017  0.137  0.002   
4     2023-02-09 16:59:58  0.175  0.019  1.016  1.316  0.017  0.143  0.001   
...                   ...    ...    ...    ...    ...    ...    ...    ...   
16247 2023-12-21 22:10:14  0.149  0.019  0.198  0.373  0.015  0.039  0.002   
16248 2023-12-21 23:13:49  0.139  0.023  0.170  0.420  0.013  0.044  0.003   
16249 2023-12-21 23:20:19  0.151  0.025  0.167  0.437  0.015  0.043  0.003   
16250 2023-12-21 23:28:37  0.138  0.024  0.173  0.379  0.013  0.039  0.002   
16251 2023-12-21 23:48:16  0.137  0.024  0.174  0.353  0.013  0.037  0.002   

                 min  
0         890.383333  
1         899.133